# Lossy Counting (Algorithm 3.2)

目的: ストリームをバケットに分割し、各要素に誤差境界 `Δ = ⌊ n/k ⌋` を付与して頻出候補を維持。

処理概要:
- 新規要素 `i` を観測したら `c[i] = 1 + Δ` として登録
- 各アイテム処理後に `Δ` が変わったバケット境界で、 `c[j] < Δ` の要素を一括削除

性質: 誤差は高々 `n/k`。MGより実装が容易で、周期的なクレンジングで高速。


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
from collections import defaultdict, Counter
import math

def lossy_counting(stream, k):
    T = {}   # element -> count (with offset)
    n = 0
    Delta = 0 # floor(n/k)
    for i in stream:
        n += 1
        if i in T:
            T[i] += 1
        else:
            T[i] = 1 + Delta
        if (n // k) != Delta:
            Delta = n // k
            # prune
            to_del = [j for j, c in T.items() if c < Delta]
            for j in to_del:
                del T[j]
    return T, n, Delta

# demo
import pandas as pd
# Updated to use the California housing data
df = pd.read_csv("/content/sample_data/california_housing_test.csv", encoding="utf-8", engine="python")
stream = df["median_income"].astype(str).tolist() # Using 'median_income' column for demonstration
k = 20
T, n, Delta = lossy_counting(stream, k)

true = Counter(stream)
cands = {u: true[u] for u in T.keys()}
print("n=", n, "Delta=", Delta, "candidates=", len(T))
print(sorted([(u, T[u], cands[u]) for u in T], key=lambda t: -cands[t[0]])[:20])
print("\nTop-20 ground truth:")
print(true.most_common(20))

n= 3000 Delta= 150 candidates= 20
[('3.2708', 150, 4), ('4.4375', 150, 3), ('3.1607', 150, 2), ('2.5417', 150, 2), ('3.3281', 150, 1), ('2.8285', 150, 1), ('2.6111', 150, 1), ('1.8633', 150, 1), ('1.1346', 150, 1), ('5.3294', 150, 1), ('3.2386', 150, 1), ('2.6923', 150, 1), ('7.1997', 150, 1), ('6.2263', 150, 1), ('5.1048', 150, 1), ('5.5867', 150, 1), ('1.179', 150, 1), ('3.3906', 150, 1), ('2.2895', 150, 1), ('8.5608', 150, 1)]

Top-20 ground truth:
[('15.0001', 9), ('3.375', 8), ('4.0', 8), ('2.125', 7), ('2.75', 7), ('3.875', 7), ('3.25', 7), ('2.625', 6), ('3.625', 6), ('2.375', 6), ('3.6875', 6), ('4.5', 6), ('4.125', 5), ('1.625', 5), ('3.9375', 5), ('2.5625', 5), ('3.75', 5), ('4.375', 5), ('2.5', 5), ('2.5833', 5)]


# Lossy Counting (Algorithm 3.2)

### コードの説明:
このコードは、ストリームデータをバケットに分割し、各要素に誤差境界 `Δ = ⌊ n/k ⌋` を付与して頻出候補を維持する「Lossy Counting アルゴリズム」を実装する。

`lossy_counting(stream, k)` 関数:

-   **入力**: データのリスト (`stream`) と、バケットのサイズに関連するパラメータ (`k`)。
-   **処理**: ストリームを順番に処理し、要素の出現回数を数える。新しい要素には `1 + Δ` のカウントを割り当てる。一定の間隔（`Δ` が更新されるタイミング）で、カウントが `Δ` より小さい要素を削除。
-   **出力**: 近似的な出現回数を持つ要素の辞書 (`T`)、ストリームの全体の要素数 (`n`)、および最終的な誤差境界 (`Delta`)。

### デモの実行:
-   サンプルデータ (`california_housing_test.csv`) を読み込み、`median_income` 列をストリームデータとして使用。
-   `lossy_counting` 関数を使って、頻出する `median_income` の値を近似的に見つける（ここでは `k=20`）。
-   正確な出現回数を `Counter` で計算し、Lossy Counting アルゴリズムの結果と比較。

### 結果の出力:
-   ストリームの要素数 (`n`)、最終的な誤差境界 (`Delta`)、およびアルゴリズムが見つけた候補の数 (`candidates`) を表示。
-   Lossy Counting アルゴリズムが見つけた候補と、それぞれの近似カウント（アルゴリズムが保持するカウント）、正確なカウントを表示。
-   正確な出現回数の上位20位 (Top-20 ground truth) を表示し、Lossy Counting の結果と比較できるようにした。

このアルゴリズムは、Misra-Griesよりも実装が容易で、周期的なクレンジングにより効率的に頻出要素を近似的に見つけたい場合に役立つ。